# Training the CHILDES Tokenizer

Using the phonemes in our CHILDES dataset, we train a tokenizer that just splits according to whitespace.

In [6]:
import pandas as pd

from datasets import load_dataset, get_dataset_config_names
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
from transformers import GPT2TokenizerFast

In [7]:
phoible = pd.read_csv('../../../data/phoible.csv')
phoible_phonemes = phoible.Phoneme.unique()

/var/folders/nz/6tzh0bsj2txd1cz18gpcms_c0000gn/T/ipykernel_87193/769076702.py:1: DtypeWarning: Columns (4,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  phoible = pd.read_csv('../../../data/phoible.csv')


In [8]:
MIN_COUNT = 10

def build_vocabulary(datasets, column='phonemized_utterance', allow_non_phoible=False):

    vocab = {'UNK' : 0, 'PAD' : 1, 'WORD_BOUNDARY' : 2, 'UTT_BOUNDARY' : 3}
    unk_tokens = []
    token_counts = {}
    for dataset in datasets:
        for line in dataset[column]:
            tokens = line.strip().split()
            for token in tokens:
                if token not in token_counts:
                    token_counts[token] = 0
                token_counts[token] += 1
        
    # Add tokens to vocab if they are not in phoible and have a count greater than MIN_COUNT
    for token, count in token_counts.items():
        if count > MIN_COUNT and token not in vocab:
            if token not in phoible_phonemes and not allow_non_phoible:
                unk_tokens.append(token)
            else:
                vocab[token] = len(vocab)

    print('Tokens not found in phoible: ', {token: token_counts[token] for token in unk_tokens})
    print('Vocab: ', vocab)
    print('Vocab size: ', len(vocab))
    return vocab

def build_phoneme_tokenizer(vocab):

    tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token='UNK'))
    tokenizer.normalizer = normalizers.Sequence(
        [normalizers.Replace("\n", " UTT_BOUNDARY"), normalizers.Strip()]
    ) # Replace newlines with utterance boundaries
    tokenizer.add_special_tokens(["UNK", "PAD"])
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

    wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer, bos_token='UTT_BOUNDARY', eos_token='UTT_BOUNDARY', pad_token='PAD', unk_token='UNK')
    return wrapped_tokenizer

# Training a Tokenizer for each language in CHILDES

We create a unique tokenizer for each language, to keep the vocabulary size appropriate for each language. For most languages we remove any tokens not found in Phoible. We do not do this for Mandarin or Cantonese as for these languages we merge the tone marker and preceding vowel into one phoneme, whereas Phoible treats tone markers as independent symbols.

In [9]:
languages = get_dataset_config_names('transformersegmentation/CHILDES')
print('Languages:', languages)
datasets = [load_dataset('transformersegmentation/CHILDES', language, split='train') for language in languages]

Languages: ['English', 'EnglishUK', 'French', 'German', 'Spanish', 'Dutch', 'Mandarin', 'Japanese', 'Cantonese', 'Estonian', 'Croatian', 'Danish', 'Basque', 'Hungarian', 'Turkish', 'Farsi', 'Icelandic', 'Indonesian', 'Irish', 'Welsh', 'Korean', 'Swedish', 'Norwegian', 'Quechua', 'Catalan', 'Italian', 'PortuguesePt', 'PortugueseBr', 'Romanian']


In [10]:
for language, dataset in zip(languages, datasets):
    print(f'\nTrainking tokenizer for {language}...')
    allow_non_phoible = language in ['Mandarin', 'Cantonese'] # For Mandarin and Cantonese, allow non-phoible tokens since we merge tone with vowels
    vocab = build_vocabulary([dataset], allow_non_phoible=allow_non_phoible)
    tokenizer = build_phoneme_tokenizer(vocab)
    tokenizer.push_to_hub(f"transformersegmentation/CHILDES-{language}-phoneme-tokenizer")
    print(f'Tokenizer for {language} pushed to the hub.')

print(f'\nTrainking tokenizer for all languages...')
vocab = build_vocabulary(datasets)
tokenizer = build_phoneme_tokenizer(vocab)
tokenizer.push_to_hub("transformersegmentation/CHILDES-phoneme-tokenizer")
print('Done.')



Trainking tokenizer for English...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'BOS': 2, 'EOS': 3, 'WORD_BOUNDARY': 4, 'UTT_BOUNDARY': 5, 's': 6, 'iː': 7, 'ð': 8, 'ɛ': 9, 'ɹ': 10, 'z': 11, 'ʌ': 12, 'f': 13, 'eɪ': 14, 'w': 15, 'ɪ': 16, 'ɡ': 17, 'l': 18, 'æ': 19, 'ɑ': 20, 'h': 21, 'ə': 22, 'ʊ': 23, 'k': 24, 'p': 25, 'uː': 26, 'b': 27, 'i': 28, 't': 29, 'aɪ': 30, 'θ': 31, 'ŋ': 32, 'j': 33, 'ɔ': 34, 'm': 35, 'ɔɪ': 36, 'n': 37, 'd': 38, 'oʊ': 39, 'aʊ': 40, 'v': 41, 'ɜː': 42, 't̠ʃ': 43, 'd̠ʒ': 44, 'ʃ': 45, 'iə': 46, 'ʒ': 47, 'ɑ̃': 48, 'r': 49, 'x': 50}
Vocab size:  51
Tokenizer for English pushed to the hub.

Trainking tokenizer for EnglishUK...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'BOS': 2, 'EOS': 3, 'WORD_BOUNDARY': 4, 'UTT_BOUNDARY': 5, 'eɪ': 6, 't̠ʃ': 7, 'w': 8, 'ɒ': 9, 't': 10, 'd': 11, 'ʌ': 12, 'z': 13, 'ð': 14, 'a': 15, 'm': 16, 'iː': 17, 'n': 18, 'ɛ': 19, 'k': 20, 's': 21, 'ɪ': 22, 'ɡ': 23, 'ʊ': 24, 'ɑː': 25, 'ɔː': 26, 'l': 27, 'ə': 28, 'ɹ'

# BPE Tokenizers for CHILDES

In [11]:
dataset = load_dataset('transformersegmentation/CHILDES', 'English', split='train')
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(), normalizers.Lowercase(), normalizers.Strip(), normalizers.StripAccents()])
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

trainer = trainers.BpeTrainer(vocab_size=8192, special_tokens=["<|endoftext|>", "<|pad|>"])
tokenizer.train_from_iterator(dataset['processed_gloss'], trainer=trainer)

tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)
tokenizer.decoder = decoders.ByteLevel()

In [12]:
example = dataset['processed_gloss'][300]
encoding = tokenizer.encode(example)
print(f'Example: {example}')
print(encoding.tokens)

Example: we're gonna get some clothes for thomas .
['Ġwe', "'re", 'Ġgonna', 'Ġget', 'Ġsome', 'Ġclothes', 'Ġfor', 'Ġthomas', 'Ġ.']


In [13]:
wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer, pad_token='<|pad|>', add_prefix_space=True)
wrapped_tokenizer.push_to_hub("transformersegmentation/CHILDES-English-BPE-gloss-tokenizer")

CommitInfo(commit_url='https://huggingface.co/transformersegmentation/CHILDES-English-BPE-gloss-tokenizer/commit/263916bb63b7e1fb06deefd8fae687d63ac0d367', commit_message='Upload tokenizer', commit_description='', oid='263916bb63b7e1fb06deefd8fae687d63ac0d367', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
tokenized = wrapped_tokenizer(example, padding='max_length', max_length=20, truncation=True)
tokenized

{'input_ids': [91, 153, 189, 168, 199, 1221, 192, 1868, 40, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [15]:
wrapped_tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

['Ġwe',
 "'re",
 'Ġgonna',
 'Ġget',
 'Ġsome',
 'Ġclothes',
 'Ġfor',
 'Ġthomas',
 'Ġ.',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>',
 '<|pad|>']